# Imports and Global Variables

In [1]:
import torch
import geoopt
import numpy as np
import pickle
import os
import sys

from scipy.linalg import expm
from logalike import Logalike

sys.path.append('../util')
from util import extract_compact_Q

In [2]:
%load_ext autoreload
%autoreload 2

path = '../data' # relative path to directory with saved data

# Running Model

In [26]:
# load true tree
fname = os.path.join(path, 'true_tree')
with open(fname, 'rb') as file:
    true_tree = pickle.load(file)
    
# load parameters for infinitesimal generator Q
fname = os.path.join(path, 'params')
with open(fname, 'rb') as file:
    params = pickle.load(file)

In [31]:
cm = true_tree.character_matrix
Q_compact = torch.tensor(extract_compact_Q(params['mutation_rate'], params['deletion_rate']))

num_mutations = torch.tensor([4, 4, 4, 4, 4, 4])
rho = torch.tensor(2, dtype=torch.float64)

# initial guess for points
num_cells = cm.shape[0]
manifold = geoopt.Lorentz(k=rho)
points = manifold.random_normal(num_cells, dtype=torch.float64)

In [32]:
l = Logalike(rho=rho,
             character_matrix=cm,
             init_points=points,
             num_mutations=num_mutations,
             S=6,)

opt = geoopt.optim.RiemannianAdam(l.parameters(), lr=1e-3)
for i in range(num_cells):
    opt.zero_grad()
    loss = l.forward(Q, i)
    loss.backward()
    opt.step()
    

tensor(123)


InvalidIndexError: (0, 0)